## Photo Tagger in SageMaker Training

We will now upload the dataset to Amazon S3...

In [2]:
import sagemaker, boto3
from sagemaker.session import Session
sess = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sess.default_bucket()
prefix = 'my-house-rooms/train'
print(bucket)

sagemaker-eu-west-1-889960878219


In [3]:
inputs = sess.upload_data(path='Split_Dataset/train', bucket=bucket, key_prefix=prefix)
print("input spec (in this case, just an S3 path): {}".format(inputs))

input spec (in this case, just an S3 path): s3://sagemaker-eu-west-1-889960878219/my-house-rooms/train


In [4]:
!pygmentize code/train.py

import argparse
import json
import logging
import os
import sys

#import sagemaker_containers
import torch
import torch.distributed as dist
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import torch.utils.data.distributed
from torchvision import datasets, transforms

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))


# Based on https://github.com/pytorch/examples/blob/master/mnist/main.py
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 32

In [9]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
    entry_point='code/train.py',
    role=role,
    py_version="py36",
    framework_version="1.8",
    instance_count=2,
    instance_type="ml.g4dn.2xlarge",
    hyperparameters={
        "epochs": 5,
        "batch-size": 32,
        "backend": "gloo"
    },
)

In [11]:
estimator.fit({"training": inputs})

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: pytorch-training-2023-02-27-13-35-19-758


2023-02-27 13:35:20 Starting - Starting the training job...
2023-02-27 13:35:36 Starting - Preparing the instances for training......
2023-02-27 13:36:29 Downloading - Downloading input data...
2023-02-27 13:37:09 Training - Downloading the training image..................
2023-02-27 13:40:01 Training - Training image download completed. Training in progress...bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-02-27 13:40:28,758 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-02-27 13:40:28,786 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-02-27 13:40:28,789 sagemaker_pytorch_container.training INFO     Invoking user training script.
2023-02-27 13:40:29,026 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "training": "/opt/ml

UnexpectedStatusException: Error for Training job pytorch-training-2023-02-27-13-35-19-758: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
Command "/opt/conda/bin/python3.6 train.py --backend gloo --batch-size 32 --epochs 5"
INFO:__main__:Initialized the distributed environment: 'gloo' backend on 2 nodes. Current host rank is 1. Number of gpus: 1
INFO:__main__:Get train data loader
INFO:__main__:Get test data loader
DEBUG:__main__:Processes 2099/4198 (50%) of train data
DEBUG:__main__:Processes 4198/4198 (100%) of test data
Traceback (most recent call last):
  File "train.py", line 256, in <module>
    train(parser.parse_args())
  File "train.py", line 147, in train
    output = model(data)
  File "/opt/conda/lib/python3.6/site-packages/torch/nn/modules/module.py", line 918, in _call_impl
    result = self.forward(*input, **kwargs)
  File "/opt/conda/lib/python3.6/site-packages/torch/nn/parallel/distributed.py", line 705, in forward
    output = self.module(*inputs[0], **kwarg

In [ ]:
predictor = estimator.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large"
)